In [1]:
# pip install yfinance backtrader numpy pandas tensorflow
import yfinance as yf
import backtrader as bt
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [29]:
def preprocess_data(data):
    # Scale open, close, high, and low prices
    data['scaled_close'] = data['Close'] / data['Open']
    data['scaled_open'] = data['Open'] / data['Open']
    data['scaled_high'] = data['High'] / data['Open']
    data['scaled_low'] = data['Low'] / data['Open']
    data['relative_volume'] = np.log(data['Volume'] / data['Volume'].shift(1))

    # Calculate Bollinger Bands
    data['middle_band'] = data['Close'].rolling(window=20).mean() / data['Close']
    data['upper_band'] = (data['middle_band'] + 2 * data['Close'].rolling(window=20).std()) / data['Close']
    data['lower_band'] = (data['middle_band'] - 2 * data['Close'].rolling(window=20).std()) / data['Close']

    # Calculate RSI
    delta = data['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    data['rsi'] = 100 - (100 / (1 + rs))

    # Calculate the log returns for volatility
    data['log_returns'] = np.log(data['Close'] / data['Close'].shift(1))

    # Remove the first rows with NaN values
    return data.dropna()

In [25]:
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=50, activation='relu', return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, activation='relu', return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(units=1, activation='linear'))

    model.compile(optimizer=Adam(lr=0.001), loss='mean_squared_error')
    return model

In [26]:
def train_lstm_model(model, train_x, train_y, epochs, batch_size):
    model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=1)
    return model

In [61]:
stock_data = yf.download("SPY", start="2023-02-14", end="2023-04-14", interval='15m')

[*********************100%***********************]  1 of 1 completed


In [62]:
preprocessed_data = preprocess_data(stock_data)
model_data = preprocessed_data.drop(columns=['Open', 'Close', 'High', "Low", 'Adj Close', 'Volume'])
model_data.shape

(1047, 10)

In [63]:
horizon = 20
test_size = 200

train_x, train_y = model_data.iloc[:-1 - test_size], model_data['log_returns'].shift(-1).dropna().iloc[:-test_size]
train_x = np.array([train_x.iloc[i-horizon:i].values for i in range(horizon, len(train_x))])
train_y = np.array(train_y[horizon:])
train_x.shape

(826, 20, 10)

In [64]:
lstm_model = create_lstm_model(train_x.shape[1:])
trained_model = train_lstm_model(lstm_model, train_x, train_y, epochs=100, batch_size=32)

training
Epoch 1/100
26/26 [==============================] - 3s 9ms/step - loss: 0.2904
Epoch 2/100
26/26 [==============================] - 0s 9ms/step - loss: 0.0122
Epoch 3/100
26/26 [==============================] - 0s 9ms/step - loss: 0.0039
Epoch 4/100
26/26 [==============================] - 0s 9ms/step - loss: 0.0021
Epoch 5/100
26/26 [==============================] - 0s 10ms/step - loss: 0.0012
Epoch 6/100
26/26 [==============================] - 0s 10ms/step - loss: 9.2395e-04
Epoch 7/100
26/26 [==============================] - 0s 10ms/step - loss: 6.4688e-04
Epoch 8/100
26/26 [==============================] - 0s 9ms/step - loss: 5.2873e-04
Epoch 9/100
26/26 [==============================] - 0s 9ms/step - loss: 3.8511e-04
Epoch 10/100
26/26 [==============================] - 0s 9ms/step - loss: 3.2612e-04
Epoch 11/100
26/26 [==============================] - 0s 10ms/step - loss: 2.5292e-04
Epoch 12/100
26/26 [==============================] - 0s 9ms/step - loss: 2.3122e

In [65]:
class CustomPandasData(bt.feeds.PandasData):
    lines = ('scaled_close', 'scaled_open', 'scaled_high', 'scaled_low', 'middle_band', 'upper_band', 'lower_band', 'rsi', 'log_returns', 'relative_volume')
    params = (
        ('scaled_close', -1),
        ('scaled_open', -1),
        ('scaled_high', -1),
        ('scaled_low', -1),
        ('upper_band', -1),
        ('lower_band', -1),
        ('middle_band', -1),
        ('rsi', -1),
        ('log_returns', -1),
        ('relative_volume', -1)
    )

In [66]:
model_data.head()

,scaled_close,scaled_open,scaled_high,scaled_low,relative_volume,middle_band,upper_band,lower_band,rsi,log_returns
Datetime,,,,,,,,,,
2023-02-14 14:15:00,1.001250,1.0,1.001881,0.999964,-0.422224,0.997511,0.009385,-0.004549,60.717814,0.001225
2023-02-14 14:30:00,1.002036,1.0,1.002618,0.999891,0.361126,0.995739,0.009640,-0.004822,61.461275,0.002034
2023-02-14 14:45:00,0.999516,1.0,1.000702,0.999032,-0.276994,0.996231,0.009613,-0.004790,69.430730,-0.000508
2023-02-14 15:00:00,0.999552,1.0,1.000702,0.998112,0.223626,0.996426,0.008933,-0.004107,77.570419,-0.000399
2023-02-14 15:15:00,1.000315,1.0,1.001187,0.999300,0.225477,0.996139,0.008838,-0.004015,75.594795,0.000254


In [69]:
class LSTMStrategy(bt.Strategy):
    params = (('model', None), ('log_threshold', 6e-3))

    def __init__(self):
        self.data_close = self.datas[0].close

    def next(self):
        # if not self.position:
        input_data = np.array([[
            self.data.scaled_close[-i],
            self.data.scaled_open[-i],
            self.data.scaled_high[-i],
            self.data.scaled_low[-i],
            self.data.relative_volume[-i],
            self.data.middle_band[-i],
            self.data.upper_band[-i],
            self.data.lower_band[-i],
            self.data.rsi[-i],
            self.data.log_returns[-i],
        ] for i in range(1, 21)]).reshape(1, 20, -1)
        prediction = self.params.model.predict(input_data, verbose=0)[0][0]
        if prediction < 0:
            print(prediction)
        if prediction > self.params.log_threshold:
            self.buy()
        if prediction < -self.params.log_threshold:
            self.sell()

In [71]:
cerebro = bt.Cerebro()

data_feed = CustomPandasData(dataname=preprocessed_data[-40:])
cerebro.adddata(data_feed)

cerebro.addstrategy(LSTMStrategy, model=trained_model)
# cerebro.addsizer(bt.sizers.AllInSizer)
cerebro.broker.set_cash(10000.0)
cerebro.broker.setcommission(commission=0.001)
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade_analyzer')
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe_ratio', riskfreerate=0.05, annualize=True, timeframe=bt.TimeFrame.Days)

starting_portfolio_value = cerebro.broker.getvalue()
print('Starting Portfolio Value: ${:.2f}'.format(starting_portfolio_value))

# Run the backtesting
results = cerebro.run()

# Get the final strategy
strategy = results[0]

ending_portfolio_value = cerebro.broker.getvalue()
print('Ending Portfolio Value: ${:.2f}'.format(ending_portfolio_value))

Starting Portfolio Value: $10000.00
[[0.00014053]]
[[0.0001392]]
[[0.00013297]]
[[0.00012546]]
[[0.00012323]]
[[0.00010403]]
[[9.571687e-05]]
[[0.00010294]]
[[0.00010401]]
[[0.00010217]]
[[0.00010091]]
[[9.8844466e-05]]
[[8.146648e-05]]
[[6.901435e-05]]
[[6.828543e-05]]
[[7.161459e-05]]
[[6.897353e-05]]
[[7.145011e-05]]
[[7.507361e-05]]
[[7.861636e-05]]
[[7.839063e-05]]
[[7.766565e-05]]
[[8.7563516e-05]]
[[9.447111e-05]]
[[0.0001004]]
[[0.00010527]]
[[0.00011317]]
[[0.0001149]]
[[0.00011494]]
[[0.00011474]]
[[0.00011675]]
[[0.00011882]]
[[0.00011956]]
[[0.00012197]]
[[0.00012907]]
[[0.00013786]]
[[0.00014157]]
[[0.00014256]]
[[0.00013985]]
[[0.00014061]]
Ending Portfolio Value: $10000.00


In [13]:

# Print the trade win rate
try:
    total_trades = strategy.analyzers.trade_analyzer.get_analysis()['total']['total']
    won_trades = strategy.analyzers.trade_analyzer.get_analysis()['won']['total']
    win_rate = won_trades / total_trades * 100
    print('Trade Win Rate: {:.2f}%'.format(win_rate))
except:
    print(strategy.analyzers.trade_analyzer.get_analysis())
# Print the Sharpe Ratio
try:
    sharpe_ratio = strategy.analyzers.sharpe_ratio.get_analysis()['sharperatio']
    print('Sharpe Ratio: {:.2f}'.format(sharpe_ratio))
except:
    print(strategy.analyzers.sharpe_ratio.get_analysis())

AutoOrderedDict([('total', AutoOrderedDict([('total', 0)]))])
OrderedDict([('sharperatio', None)])
